In [ ]:
%matplotlib inline

from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import time

from numpy import diag, exp, log, sqrt, trace
from numpy.linalg import eig, inv, norm, solve, svd
from scipy.linalg import expm, solve_lyapunov, solve_sylvester, sqrtm
from scipy.sparse import diags

$$\mathrm dX_t = (AX_t + B u_t)\mathrm d t + N X_{t-\tau}\mathrm dW_t$$

In [ ]:
d = 40
r = 10

X_0 = np.array(([0.1] * r + (d - r) * [0.1]))[:, np.newaxis]

A = - (np.diag([1] * r + [1] * (d - r)) + 0.01 * np.random.randn(d, d))
B = np.diag([1] * r + [1] * (d - r)) + 0.01 * np.random.randn(d, d)
N = np.diag([1] * r + [1] * (d - r)) + 0.01 * np.random.randn(d, d)

if ~np.all(np.real(eig(A)[0]) < 0):
    print('not all eigenvalues are negative')

C = np.diag([1] * r + [.01] * (d - r))
#B_in = 0.1 * np.ones([d, d])
B_in = X_0.dot(X_0.T)
#B_in = 0.1 * np.ones([d])

def u_t(x, t):
    return np.sin(20 * t) * np.ones(x.shape)

def history_function(t, x):
    return np.zeros(x.shape)

In [ ]:
I = 800

Q_i = np.zeros([I + 1, d, d])
Q_i[0, :, :] = solve_lyapunov(A.T, -C.T.dot(C))
for i in range(I):
    Q_i[i + 1, :, :] = solve_lyapunov(A.T, -N.T.dot(Q_i[i, :, :]).dot(N))
Q = np.sum(Q_i, 0)

P_i = np.zeros([I + 1, d, d])
P_i[0, :, :] = solve_lyapunov(A, -B.dot(B.T))
for i in range(I):
    P_i[i + 1, :, :] = solve_lyapunov(A, -N.dot(P_i[i, :, :]).dot(N.T))
P = np.sum(P_i, 0)
P += B_in.dot(B_in.T)

if np.sum(P_i[-1, :, :]) > 1e-6 or np.sum(Q_i[-1, :, :]) > 1e-6:
    print('Lyapunov solver did not converge')
    
#K = sqrtm(P)
#L = sqrtm(Q)

V_P, sigmas_P, U_P_T = svd(P)
K = V_P.dot(sqrt(diag(sigmas_P))).dot(U_P_T)

V_Q, sigmas_Q, U_Q_T = svd(Q)
L = V_Q.dot(sqrt(diag(sigmas_Q))).dot(U_Q_T)

V, sigmas, U_T = svd(K.T.dot(L))
Sigma = np.diag(sigmas)
Sigma_inv_05 = np.diag(1 / sqrt(sigmas))

T = Sigma_inv_05.dot(U_T).dot(L.T)
T_inv = K.dot(V).dot(Sigma_inv_05)
A_ = T.dot(A).dot(T_inv)
B_ = T.dot(B)
N_ = T.dot(N).dot(T_inv)
C_ = C.dot(T_inv)

plt.scatter(range(1, d + 1), sigmas) # (Hankel singular values)
plt.yscale('log');
plt.ylim(0.8 * min(sigmas), 1.2 * max(sigmas));

In [ ]:
r = 8

A_11 = A_[:r, :r]
A_12 = A_[:r, r:]
A_21 = A_[r:, :r]
A_22 = A_[r:, r:]
B_1 = B_[:r, :]
B_2 = B_[r:, :]
N_11 = N_[:r, :r]
N_12 = N_[:r, r:]
N_21 = N_[r:, :r]
N_22 = N_[r:, r:]
C_1 = C_[:, :r]
C_2 = C_[:, r:]

# Balanced truncation
A_r = A_11
B_r = B_1
N_r = N_11
C_r = C_1

In [ ]:
T_end = 2
tau = 0.1
delta_t = 0.001
K = 15
N_end = int(T_end / delta_t)
tau_n = int(tau / delta_t)

u_L2 = np.sqrt(T_end / 2 - np.sin(2 * 20 * T_end) / (4 * 20))

X = np.zeros([N_end + 1, d, K])
X_r = np.zeros([N_end + 1, r, K])
X[0, :, :] =  X_0.repeat(K, 1) # np.ones([d, K])
X_r[0, :, :] = T.dot(X_0.repeat(K, 1))[:r, :]

np.random.seed(42)

for n in range(N_end):
    xi = np.random.randn(1, K)
    u = u_t(X[n, :, :], n * delta_t)
    u_r = u
    if n >= tau_n:
        X_delay = X[n - tau_n, :, :] 
        X_r_delay = X_r[n - tau_n, :, :] 
    else:
        X_delay = history_function(n * delta_t, X[n, :, :])
        X_r_delay = history_function(n * delta_t, X_r[n, :, :])
        
    X[n + 1, :, :] = (X[n, :, :] + (np.dot(A, X[n, :, :]) + np.dot(B, u)) * delta_t 
                      + np.dot(N, X_delay * xi) * np.sqrt(delta_t))
    X_r[n + 1, :, :] = (X_r[n, :, :] + (np.dot(A_r, X_r[n, :, :]) + np.dot(B_r, u_r)) * delta_t 
                      + np.dot(N_r, X_r_delay * xi) * np.sqrt(delta_t))
    
Y = C.dot(X)
Y_r = C_r.dot(X_r)

L2_error = np.sqrt(np.sum(np.sum((Y - np.real(Y_r))**2, 0)) * delta_t)

In [ ]:
t_val = np.linspace(0, T_end, N_end + 1)

COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:olive',
          'tab:pink', 'tab:gray', 'tab:brown', 'tab:cyan']

k = 3
j = 3

plt.title(r'$d = %d, r = %d, \Delta t = %.1e$' % (d, r, delta_t))
for i, k in enumerate([0]):#enumerate(np.random.choice(range(d), 1, replace=False)):
    plt.plot(t_val, Y[j, :, k], color=COLORS[2*i])
    plt.plot(t_val, Y_r[j, :, k], '-', color=COLORS[2*i+1]);#, markevery=5

In [ ]:
Y_2 = deepcopy(Y)
Y_2_r = deepcopy(Y_r)

In [ ]:
L2_errors = []
bounds_thm3 = []
    
for r in range(1, d):
    print(r)
    A_11 = A_[:r, :r]
    A_12 = A_[:r, r:]
    A_21 = A_[r:, :r]
    A_22 = A_[r:, r:]
    B_1 = B_[:r, :]
    B_2 = B_[r:, :]
    N_11 = N_[:r, :r]
    N_12 = N_[:r, r:]
    N_21 = N_[r:, :r]
    N_22 = N_[r:, r:]
    C_1 = C_[:, :r]
    C_2 = C_[:, r:]

    # Balanced truncation
    A_r = A_11
    B_r = B_1
    N_r = N_11
    C_r = C_1
    
    tau = 0.1
    delta_t = 0.001
    K = 10000
    N_end = int(T_end / delta_t)
    tau_n = int(tau / delta_t)

    X = np.zeros([N_end + 1, d, K])
    X_r = np.zeros([N_end + 1, r, K])

    X[0, :, :] =  X_0.repeat(K, 1) # np.ones([d, K])
    X_r[0, :, :] = T.dot(X_0.repeat(K, 1))[:r, :]

    for n in range(N_end):
        xi = np.random.randn(1, K)
        u = u_t(X[n, :, :], n * delta_t)
        u_r = u
        if n >= tau_n:
            X_delay = X[n - tau_n, :, :] 
            X_r_delay = X_r[n - tau_n, :, :] 
        else:
            X_delay = history_function(n * delta_t, X[n, :, :])
            X_r_delay = history_function(n * delta_t, X_r[n, :, :])

        X[n + 1, :, :] = (X[n, :, :] + (np.dot(A, X[n, :, :]) + np.dot(B, u)) * delta_t 
                          + np.dot(N, X_delay * xi) * np.sqrt(delta_t))
        X_r[n + 1, :, :] = (X_r[n, :, :] + (np.dot(A_r, X_r[n, :, :]) + np.dot(B_r, u_r)) * delta_t 
                          + np.dot(N_r, X_r_delay * xi) * np.sqrt(delta_t))
    
    Y = C.dot(X)
    Y_r = C_r.dot(X_r)
    
    L2_error = np.sqrt(np.mean(np.sum(np.sum((Y - Y_r)**2, 0), 0) * delta_t))
    L2_errors.append(L2_error)
    
    A_hat = np.zeros([d + r, d + r])
    A_hat[0:d, 0:d] = A
    A_hat[d:d + r, d:d + r] = A_r
    B_hat = np.concatenate([B, B_r])
    B_in_hat = np.concatenate([B_in, T.dot(B_in)[:r, :]])
    C_hat = np.concatenate([C, -C_r], axis=1)
    N_hat = np.zeros([d + r, d + r])
    N_hat[0:d, 0:d] = N
    N_hat[d:d + r, d:d + r] = N_r
    
    I = 800
    Q_hat_i = np.zeros([I + 1, d + r, d + r])
    Q_hat_i[0, :, :] = solve_lyapunov(A_hat.T, -C_hat.T.dot(C_hat))
    for i in range(I):
        Q_hat_i[i + 1, :, :] = solve_lyapunov(A_hat.T, -N_hat.T.dot(Q_hat_i[i, :, :]).dot(N_hat))
    Q_hat = np.sum(Q_hat_i, 0)

    I = 800
    P_hat_i = np.zeros([I + 1, d + r, d + r])
    P_hat_i[0, :, :] = solve_lyapunov(A_hat, -B_hat.dot(B_hat.T))
    for i in range(I):
        P_hat_i[i + 1, :, :] = solve_lyapunov(A_hat, -N_hat.dot(P_hat_i[i, :, :]).dot(N_hat.T))
    P_hat = np.sum(P_hat_i, 0)
    P_hat += B_in_hat.dot(B_in_hat.T)

    if np.sum(P_hat_i[-1, :, :]) > 1e-6 or np.sum(Q_hat_i[-1, :, :]) > 1e-6:
        print('Lyapunov solver did not converge')

    #K_hat = sqrtm(P_hat)
    #L_hat = sqrtm(Q_hat)

    V_P_hat, sigmas_P_hat, U_P_hat_T = svd(P_hat)
    K_hat = V_P_hat.dot(sqrt(diag(sigmas_P_hat))).dot(U_P_hat_T)

    V_Q_hat, sigmas_Q_hat, U_Q_hat_T = svd(Q_hat)
    L_hat = V_Q_hat.dot(sqrt(diag(sigmas_Q_hat))).dot(U_Q_hat_T)

    V_hat, sigmas_hat, U_T_hat = svd(K_hat.T.dot(L_hat))
    Sigma_hat = np.diag(sigmas_hat)
    Sigma_inv_05 = np.diag(1 / sqrt(sigmas_hat))
    
    bounds_thm3.append(sum(sigmas_hat) * (np.sqrt(np.sum(X[0, :, 0]**2)) + 2 * u_L2))

    print('error: %.3e, bound: %.3e' % (L2_errors[-1], bounds_thm3[-1]))

In [ ]:
COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:olive',
          'tab:pink', 'tab:gray', 'tab:brown', 'tab:cyan']

fig, ax = plt.subplots(1, 3, figsize=(15, 3.5))
ax[0].set_title('Errors of reduced systems')
ax[0].scatter(np.arange(1, d), L2_errors, label=r'$L^2$ error')
ax[0].scatter(np.arange(1, d), bounds_thm3, label='bound')
ax[0].set_yscale('log')
ax[0].set_ylim(0.5 * min(L2_errors), 2 * max(bounds_thm3));
ax[0].set_xlabel(r'$r$')
ax[0].legend()

t_val = np.linspace(0, T_end, N_end + 1)

j = 3

ax[1].set_title(r'Full and reduced trajectories, $d = %d, r = %d$' % (d, 2))
for i, k in enumerate([1]):
    ax[1].plot(t_val, Y_2[j, :, k], color=COLORS[i], label='full')
    ax[1].plot(t_val, Y_2_r[j, :, k], '-', color=COLORS[i + 1], label='reduced');
ax[1].set_xlabel(r'$t$')
ax[1].legend()

ax[2].set_title(r'Full and reduced trajectories, $d = %d, r = %d$' % (d, 8))
for i, k in enumerate([1]):
    ax[2].plot(t_val, Y[j, :, k], color=COLORS[i])
    ax[2].plot(t_val, Y_r[j, :, k], '-', color=COLORS[i + 1]);
ax[2].set_xlabel(r'$t$');
    
#fig.savefig('img/geometric_brownian_motian_2.pdf')